In [20]:
import datetime
import json
from pathlib import Path
import importlib

from lxml import etree
from nameparser import HumanName

import doi
import voyager

In [36]:
with open("all_matched_up.json", "r", encoding="utf8") as fh:
    matches = json.load(fh)

In [4]:
articles = [etree.HTML(open(p, "r", encoding="utf8").read()) for p in Path("air_pages/").glob("*.html")]
idnums = [p.name.split(".")[0] for p in Path("air_pages/").glob("*.html")]
        
    

In [5]:
contributors = voyager.get_article_contributors_from_etrees(articles)

In [23]:
years = voyager.get_article_publication_years_from_etrees(articles)
years[:25]

['2011',
 '2011',
 '2011',
 '2011',
 '2011',
 '2011',
 '2011',
 '2011',
 '2011',
 '2011',
 '2011',
 '2011',
 '2011',
 '2011',
 '2011',
 '2011',
 '2011',
 '2011',
 '2011',
 '2011',
 '2011',
 '2011',
 '2011',
 '2011',
 '2011']

In [27]:
def contributors_to_crossref(contributors):
    formatted_contributors = []
    for c in contributors:
        name = HumanName(c.split("(")[0])
        crossref_name = {"given_name": " ".join([name.first, name.middle]).strip(), "surname": name.last}
        formatted_contributors.append(crossref_name)
        
    return formatted_contributors

In [28]:
fc = [contributors_to_crossref(c) for c in contributors]

In [29]:
fc[0]

[{'given_name': 'John F.', 'surname': 'Patience'},
 {'given_name': 'Amanda', 'surname': 'Chipman'}]

In [30]:
len(idnums)

1442

In [31]:
len(contributors)

1442

In [32]:
len(fc)

1442

In [33]:
icy = list(zip(idnums, fc, years))

In [34]:
icy[0]

('5635',
 [{'given_name': 'John F.', 'surname': 'Patience'},
  {'given_name': 'Amanda', 'surname': 'Chipman'}],
 '2011')

In [37]:
matches[0]

['Carbon Monoxide Packaging for Fresh Pork',
 'Carbon Monoxide Packaging for Fresh Pork',
 'ASL R1848',
 'ASL R1848',
 'https://doi.org/10.31274/ans_air-180814-70',
 'https://www.iastatedigitalpress.com/air/article/id/6401/']

In [38]:
for i in icy:
    for x, m in enumerate(matches):
        if i[0] in m[-1]:
            matches[x].extend(i[1:])
            break
            

In [39]:
matches[0]

['Carbon Monoxide Packaging for Fresh Pork',
 'Carbon Monoxide Packaging for Fresh Pork',
 'ASL R1848',
 'ASL R1848',
 'https://doi.org/10.31274/ans_air-180814-70',
 'https://www.iastatedigitalpress.com/air/article/id/6401/',
 [{'given_name': 'Travis R.', 'surname': 'Krause'},
  {'given_name': 'Joseph G.', 'surname': 'Sebranek'},
  {'given_name': 'Mark S.', 'surname': 'Honeyman'}],
 '2004']

In [52]:
importlib.reload(doi)
root = doi.CrossrefXML()
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
depositor = doi.Depositor(timestamp, timestamp)
reports = [
    doi.ItemMetadata(
        contributors=m[6],
        date={"year": m[7], "month": "01", "day": "01"},
        doi=m[4][16:],
        kind="report",
        media_type="online",
        resource=m[5],
        title=m[0],
    )
    for m
    in matches
]

In [53]:
root.insert_depositor(depositor.to_xml())
for r in reports:
    root.insert_item_metadata(r.to_xml())
    
with open("air_doi_update.xml", "w", encoding="utf8") as fh:
    fh.write(
        etree.tostring(
            root.to_xml(), xml_declaration=True, encoding="UTF-8", pretty_print=True
        ).decode("utf-8")
    )
    